In [32]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as sw
import gensim
import numpy as np

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     C:\Users\Alberto\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping models\word2vec_sample.zip.


In [24]:
stopWords = set(sw.words("english"))

In [27]:
def lesk(word, sentence):
    senses =wn.synsets(word)
    best_sense= senses[0]
    max_overlap = 0
    context = set(nltk.word_tokenize(sentence))
    context = context.difference(stopWords)
    for sense in senses[1:]:
        data = set(nltk.word_tokenize(sense.definition()+" "+" ".join( sense.examples())))
        signature = data.difference(stopWords)
        overlap = len(context.intersection(signature))
        if overlap >max_overlap:
            max_overlap = overlap
            best_sense = sense
            
    return best_sense

In [28]:
lesk("bank","Yesterday I went to the bank to withdraw the money and the credit card did not work")

Synset('depository_financial_institution.n.01')

In [29]:
def distanceCosine(a,b):
    dot = np.dot(a, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    cos = dot / (norma * normb)
    return cos

In [30]:
from nltk.data import find

In [33]:
# Cargar el modelo de embeding pre-entrenados del NLTK
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [34]:
vector_mean =  model.vectors.mean(axis=0)

In [38]:
def wsd_embeddings(word, sentence):
    distances = []
    senses =wn.synsets(word)
    context = set(nltk.word_tokenize(sentence))
    context = context.difference(stopWords)
    matrixContext = np.zeros((len(context),300))
    i= 0
    for word in context:
        if word in model.key_to_index:
            matrixContext[i]=model[word]
        else:
            matrixContext[i]=vector_mean
        i+=1
    vectorcontext = matrixContext.mean(axis=0)
    for sense in senses:
        data = set(nltk.word_tokenize(sense.definition()+" "+" ".join( sense.examples())))
        signature = data.difference(stopWords)
        matrixSiganture = np.zeros((len(signature),300))
        i= 0
        for word in signature:
            if word in model.key_to_index:
                matrixSiganture[i]=model[word]
            else:
                matrixSiganture[i]=vector_mean
            i+=1
        vectorSignature = matrixSiganture.mean(axis=0)

        distances.append((sense,distanceCosine(vectorcontext,vectorSignature)))
    
            
    return sorted(distances, key=lambda t:t[1],reverse=True)[0]

In [39]:
t = wsd_embeddings("bank", "Yesterday I went to the bank to withdraw the money and the credit carddid not work")

In [42]:
t[0].examples()

['She deposits her paycheck every month']